# 04-01 Visualisierung-Demo

## Hinweise zur Übung

Diese Demo schließt an die beiden vorherigen SQL- & Pandas Übungen an und zeigt, wie abgefragten Daten in Form von Diagrammen und Geovisualisierungen grafisch aufbereitet werden können.

Für Diagramme wird dabei das *Altair*-Package verwendet, das in Python sehr hochwertige und vom Konzept her mit dem ggplot2-Paket in R vergleichbare Diagramme generieren kann.

Für die Geovisualisierungen verwenden wir das *Folium*-Package, das die interaktive Darstellung von Landkarten & darauf platzierten Markern über die Leaflet-Bibliothek (auch in R verfügbar) realisiert.

Wir erstellen die Abfragen in diesem Notebook direkt in SQL (vgl. Übung 02) und laden die bereits gefilterten/aggregierten Ergebnisse in Dataframes (vgl. Übung 03). Für einzelne Transformationen (z.B. Pivotieren vom "wide"- ins "long"-Format) nutzen wir Pandas-Methoden.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql pandas altair folium

In [ ]:
import os
import sys
import urllib.request
import gzip
import shutil
import pandas as pd
import altair as alt
import folium
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# Konfiguration
base_url_quellen   = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"
base_url_reporting = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/dwh/reporting.sqlite.gz", base_url_reporting + "reporting.sqlite.gz")

# Die Sqlite-Datenbank ist aufgrund ihrer Größe gezipped und muss vor der Nutzung noch entpackt werden
with gzip.open(base_url_reporting + "reporting.sqlite.gz", "rb") as f_in:
    with open(base_url_reporting + "reporting.sqlite", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_reporting      = base_url_reporting + "reporting.sqlite"

db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_reporting

## Einfache Diagramme mit Altair

In diesem Abschnitt sollen einfache Abfragen mit verschiedenen Diagrammtypen von Altair dargestellt werden, um in die Vorgehensweise einzuführen.


### Säulen- & Balkendiagramme

Der Aufbau von Diagrammen erfolgt bei Altair schrittweise, ausgehend von einem allgemeinen Diagramm-Objekt, dem nach und nach die darzustellenden Daten, Darstellungsform (Mindestangaben) sowie optional Details zur konkreten Visualisierung (z.B. Größen, Farben, Achsenbeschriften) zugewiesen werden.

Zuerst lesen wir einen einfachen Datensatz (Geschlechtsverteilung der Patient:innen im DWH) in einen Dataframe:

In [ ]:
# Geschlechtsverteilung der Patienten in der Tabelle D_PATIENT aggregieren
sql = """
SELECT patient_geschlecht,
       COUNT(*) AS n
  FROM d_patient
 GROUP BY patient_geschlecht
"""
resultset = %sql $db_url_reporting $sql
df_geschlecht = resultset.DataFrame()
df_geschlecht.head()

Für das *Altair*-Package wurde beim Import das Alias `alt` festgelegt, so dass der Package-Name beim Aufruf nicht ausgeschrieben werden muss.
Mit der `Chart()`-Methode wird zunächst ein Diagramm-Objekt erzeugt und ein Dataframe als Datenquelle festgelegt:

`alt.Chart(dataframe)`

Über verschiedene "mark"-Methoden wird dann die Art der Darstellung festgelegt, hier als Säulendiagramm (`mark_bar()`):

`alt.Chart(dataframe).mark_bar()`

Mit der `encode()`-Methode wird anschließend festgelegt, welche Spalten des Dataframes auf die X- bzw. Y-Achse abgebildet werden sollen:

```
alt.Chart(dataframe).mark_bar().encode(
  x = alt.X("spalte1"),
  y = alt.Y("spalte2")
)
```

Mit dieser einfachsten Form kann bereits ein Diagramm dargestellt werden, bei dem Altair Defaultwerte für alle restlichen Attribute einsetzt (beispielsweise die Spaltennamen als Achsenbeschriftungen übernimmt):



In [ ]:
# Säulendiagramm der Geschlechtsverteilung mit Default-Einstellungen
alt.Chart(df_geschlecht) \
  .mark_bar() \
  .encode(
    x = alt.X("patient_geschlecht"),
    y = alt.Y("n")
  )

Zur Umstellung von einem Säulen auf ein Balkendiagramm reicht es aus, die X- & Y-Achsen zu vertauschen:

In [ ]:
# Darstellung als Balkendiagramm durch Vertauschen der X- & Y-Zuordnungen
alt.Chart(df_geschlecht) \
  .mark_bar() \
  .encode(
    x = alt.X("n"),
    y = alt.Y("patient_geschlecht")
  )

Die Eigenschaften des Diagramms können detailliert festgelegt werden, indem z.B. bei der X-Achse in der `alt.X`-Funktion weitere Parameter spezifiziert werden:

`alt.X('Spaltenname', axis=alt.Axis(title='Achsenbeschriftung'))`

Mit der `properties()`-Methode können übergreifende Eigenschaften des Diagramms wie der Titel sowie die Größe festgelegt werden.

In [ ]:
# Säulendiagramm mit festgelegter Größe, Farben, Titel & Achsenbeschriftung
alt.Chart(df_geschlecht) \
  .mark_bar() \
  .encode(
    x     = alt.X("patient_geschlecht", axis=alt.Axis(title="Geschlecht")),
    y     = alt.Y("n", axis=alt.Axis(title="Anzahl")),
    color = alt.Color("patient_geschlecht", legend=alt.Legend(title="Geschlecht"))
  ) \
  .properties(
    title  = "Geschlechtsverteilung",
    width  = 200,
    height = 250
  )

### Pie- & Donut-Charts

Durch Austausch der `mark_bar()`-Methode gegen `mark_arc()` wird statt eines Balkendiagramms ein Tortendiagramm gezeichnet.
Statt den X/Y-Achsen (die im Tortendiagramm keine Bedeutung haben), wird ein neuer Parameter `theta` angegeben, der aus der im Diagramm visualisierten Kennzahl die Winkel & Breiten der einzelnen Tortenabschnitte ableitet.

In [ ]:
# Tortendiagramm für die Geschlechtsverteilung
alt.Chart(df_geschlecht) \
  .mark_arc() \
  .encode(
    theta = alt.Theta("n"),
    color = alt.Color("patient_geschlecht", legend=alt.Legend(title="Geschlecht"))
  )

Diagramme können in Altair auch zunächst in mehreren Teilen separat erzeugt und über eine einfache "Addition" zusammengesetzt werden. Im folgenden Beispiel wird zunächst ein zugrundeliegendes Diagrammobjekt "base" erzeugt, das gemeinsame Eigenschaften für die daraus abgeleiteten Teildiagramme bestimmt (in diesem Fall die Zuordnung der Kennzahl zum `theta`-Parameter sowie die Zuordnung der Farben). Wichtig ist hierbei der Parameter `stack=True`, der sicherstellt, dass die Proportionen für alle abgeleiteten Teildiagramme beibehalten werden.

Aus dem Base-Objekt wird anschließend mit der `mark_arc()`-Methode ein Tortendiagramm erzeugt. Danch wird ein separates Diagramm mit den Wert-Labels erzeugt. Durch Angabe verschiedener Radii für das Tortendiagramm und die Beschriftungen wird sichergestellt, dass sie sich nicht überlappen.

Im letzten Schritt werden die beiden Teildiagramme über eine Addition `+` zusammengefügt und ausgegeben.

In [ ]:
# Tortendiagramm ergänzt um Labels
base = alt.Chart(df_geschlecht) \
  .encode(
    theta = alt.Theta("n", stack=True),
    color = alt.Color("patient_geschlecht", legend=alt.Legend(title="Geschlecht"))
  )

# Tortendiagramm generieren
pie    = base.mark_arc(radius=120)

# Wert-Labels für die Tortenstücke generieren
labels = base.mark_text(radius=140, size=10).encode(text="n")

# Diagramme kombinieren und ausgeben
pie + labels

Die `mark_arc()`-Methode unterstützt auch Donut-Charts (Ringdiagramme), indem ein zusätzliche Parameter `innerRadius` angegeben wird, der die Größe des weißen Innenbereichs angibt.

In [ ]:
# Vorherige Grafik als Donut-Diagramm
base = alt.Chart(df_geschlecht) \
  .encode(
    theta = alt.Theta("n", stack=True),
    color = alt.Color("patient_geschlecht", legend=alt.Legend(title="Geschlecht"))
  )

pie    = base.mark_arc(radius=120, innerRadius=50)
labels = base.mark_text(radius=140, size=10).encode(text="n")

pie + labels

### Linien- & Flächendiagramme

Verlaufsdaten werden typischerweise in Form von Linien oder Flächendiagrammen dargestellt.

Wir laden hierfür zunächst einen neuen Datensatz mit der Fallzahlentwicklung der beiden Standorte Bielefeld und Mannheim von 1980 bis heute, aggregiert pro Jahr:

In [ ]:
# Fallzahl pro Standort und Jahr ab 01.01.1980 auslesen
sql = """
SELECT org.standort_name,
       SUBSTR(fal.aufnahme_datum, 1, 4) AS jahr,
       COUNT(*) AS n
  FROM f_faelle fal
  JOIN d_orga   org ON fal.einrichtung_id = org.einrichtung_id
 WHERE fal.aufnahme_datum >= '1980-01-01'
 GROUP BY org.standort_name,
       SUBSTR(fal.aufnahme_datum, 1, 4)
"""
resultset = %sql $db_url_reporting $sql
df_fallzahlen = resultset.DataFrame()
df_fallzahlen.head()

Mit der Methode `mark_line()` können Liniendiagramme dargestellt werden:

In [ ]:
alt.Chart(df_fallzahlen) \
  .mark_line() \
  .encode(
    x     = alt.X("jahr"),
    y     = alt.Y("n"),
    color = alt.Color("standort_name")
  ) \
  .properties(
    title  = "Fallzahlen",
    width  = 600,
    height = 250
  )

Durch einfachen Austausch gegen die Methode `mark_area()` wird ein gestapeltes Flächendiagramm generiert, in dem die Gesamtzahl der Fälle pro Monat sowie der jeweilige Anteil pro Standort besser erkennbar sind:

In [ ]:
alt.Chart(df_fallzahlen) \
  .mark_area() \
  .encode(
    x     = alt.X("jahr"),
    y     = alt.Y("n"),
    color = alt.Color("standort_name")
  ) \
  .properties(
    title  = "Fallzahlen",
    width  = 600,
    height = 250
  )

### Scatterplots

Bei Scatterplots werden Daten als Punkte oder Kreise in ein X/Y-Koordinatensystem eingetragen. Durch die Nutzung beider Achsen, der Farbe und ggf. der Größe der Kreise kann eine größere Menge von Kennzahlen gleichzeitig in einem Diagramm dargestellt werden, als es über Balken-, Linien- oder Flächendiagramme möglich ist.

Wir fragen hierfür zunächst einen komplexeren Datensatz ab, der Angaben zu den Hauptdiagnosen der Fälle im Kurs-DWH wiedergibt:
* Kapitelname der Hauptdiagnose im ICD-Katalog
* Jahr der Aufnahme
* Durchschnittliches Alter in Jahren bei Aufnahme
* Anzahl der Fälle

Um eine überschaubare Anzahl von Elementen im Diagramm zu sehen, filtern wir den Datensatz auf Diagnosen mit mindestens 10 Fällen pro Datenpunkt.


In [ ]:
# Kennzahlen zu den dokumentierten Diagnosen auswerten
sql = """
SELECT dkt.kapitel_name,
       SUBSTR(fal.aufnahme_datum, 1, 4) AS jahr,
       AVG(fal.aufnahmealter_jahre)     AS aufnahmealter_avg,
       COUNT(*)                         AS fallzahl
  FROM f_faelle   fal
  JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
 WHERE dkt.kapitel_name <> ''
 GROUP BY dkt.kapitel_name,
       SUBSTR(fal.aufnahme_datum, 1, 4)
HAVING COUNT(*) >= 10
"""
resultset = %sql $db_url_reporting $sql
df_diagnoseverlauf = resultset.DataFrame()
df_diagnoseverlauf.head()

Scatterplots werden in Altair mit der Methode `mark_point()` erzeugt. Wir nutzen die bekannten Attribute x, y und color für die Zuweisung der X-Achse (Aufnahmejahr), Y-Achse (mittleres Alter bei Aufnahme) und Farbe (ICD-Kapitel). Als zusätzliches Attribut definieren wir die Größe der Kreise mit dem `size`-Parameter (Fallzahl).

Da der Datentyp vor allem bei Dataframes im "long"-Format nicht immer automatisch erkennbar ist, erlaubt Altair die explizite Angabe eines Datentyps für die einbezogenen Spalten über ein Suffix mit einem Doppelpunkt `:`. Hierbei können u.a. folgende Typen angegeben werden:
* Q: Quantitative (numerischer Wert)
* T: Temporal (Datums- und/oder Zeitangabe)
* N: Nominal (Ausprägungsliste ohne Reihenfolge)
* O: Ordinal (Ausprägungsliste mit Reihenfolge)

Im Beispiel unten wird das Jahr als temporal (T) definiert, das Aufnahmealter und die Fallzahl als quantitativ (Q) und das ICD-Kapitel als nominal (N).

Das Attribut `tooltip` zeigt einen Mouseover-Effekt mit Angaben zum Diagnosekapitel, Jahr, Fallzahl und Aufnahmealter.




In [ ]:
alt.Chart(df_diagnoseverlauf) \
  .mark_point() \
  .encode(
    x     = alt.X("jahr:T", axis=alt.Axis(title="Jahr")),
    y     = alt.Y("aufnahmealter_avg:Q", axis=alt.Axis(title="Aufnahmealter")),
    size  = alt.Size("fallzahl:Q"),
    color = alt.Color("kapitel_name:N"),
    tooltip = ['kapitel_name', 'jahr', 'fallzahl', 'aufnahmealter_avg']
  ) \
  .properties(
    title  = "Diagnosen",
    width  = 500,
    height = 500
  )

## Statistische Diagramme mit Altair

Bei den bisher vorgestellten "einfachen" Diagrammen werden die Daten unverändert visualisiert. Bei den folgenden statistischen Diagrammen erfolgt eine Vorverarbeitung (z.B.Binning bei Histogrammen) oder Ableitung von relevanten Intervallen (z.B. Quantile bei Boxplots), die eine bessere Beurteilung der Verteilungsmuster der Daten erlauben.

### Histogramme

Im folgenden sollen Histogramme am Beispiel der Altersverteilung von Hauptdiagnosen im Kurs-DWH dargestellt werden.

Wir laden dafür zunächst einen Rohdatensatz aus nicht aggregierten Aufnahmealtersangaben mit Bezug auf das jeweilige Kapitel des ICD10-Diagnosecodes. Der Datensatz liegt also anders als in den vorherigen Beispielen nicht bereits aggregiert vor, sondern enthält Einzelangaben zum Alter pro Behandlungsfall mit dem Kapitel der dokumentierten Hauptdiagnose.

🛑 Caveat: Altair generiert zur Visualisierung eine JavaScript-Datei, die sowohl den Code zur Erzeugung des Diagramms als auch die benötigten Daten (im JSON-Format) enthält. Aufgrund der Größe dieser Datei gibt es eine Begrenzung für die Anzahl der mit Altair darstellbaren Datensätze (defaultmäßig bis zu 5.000 Zeilen).

In [ ]:
# ...
sql = """
SELECT dkt.kapitel_id,
       dkt.kapitel_name,
       fal.aufnahmealter_jahre
  FROM f_faelle   fal
  JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
 WHERE dkt.kapitel_name <> ''
"""
resultset = %sql $db_url_reporting $sql
df_aufnahmealter = resultset.DataFrame()
df_aufnahmealter.head()

Histogramme werden in Pandas wie normale Säulendiagramme mit der `mark_bar()`-Methode erzeugt, jedoch wird beim Encoding der X-Achse das Attribut `bin=True` angegeben. Hierdurch werden die Einzelwerte in Klassen zusammengefasst. Durch die Angabe von `count()` beim Encoding der Y-Achse wird als Kennzahl die Anzahl der pro Klasse enthaltenen Datensätze verwendet.

Für das folgende Diagramm filtern wir den geladenen Dataframe nur auf das ICD10-Kapitel E (Metabolische Erkrankungen).

In [ ]:
alt.Chart(df_aufnahmealter.query("kapitel_id=='E'")) \
  .mark_bar() \
  .encode(
    x = alt.X("aufnahmealter_jahre:Q", bin=True),
    y = alt.Y('count()')
  )

Sofern nur das Attribut `bin=true()` gesetzt ist, bestimmt Altair eigenständig die Anzahl der Klassen. mit der Funktion `alt.Bin(maxbins=<Anzahl>)` kann die Anzahl der Kalssen beeinflusst werden, um z.B. eine feingranularere Darstellung zu erhalten:

In [ ]:
alt.Chart(df_aufnahmealter.query("kapitel_id=='E'")) \
  .mark_bar() \
  .encode(
    x = alt.X("aufnahmealter_jahre:Q", bin=alt.Bin(maxbins=40)),
    y = alt.Y('count()')
  )

Auch bei Histogrammen können Farben benutzt werden, um verschiedene Gruppen von Daten in einem Diagramm darzustellen. Im folgenden Beispiel passen wir die Filterung an, um neben den metabolischen Erkrankungen (ICD10-Kapitel E) auch noch das Kapitel P (Erkrankungen der Perinatalperiode) aufzunehmen und im Vergleich auszuwerten.

Wir setzen das Attribut `color` im Encoding und ordnen ihm den ICD10-Kapitelnamen zu, um die beiden Kapitel im Diagramm differenzieren zu können. Um beide Verteilungen nebeneinander interpretieren zu können, sollten sie nicht (wie beim Flächendiagramm oben) aufeinander gestapelt sein, sondern sich überlagern. Hierzu setzen wir im Encoding der Y-Achse die Option `stack=None` sowie eine halbtransparente Darstellung durch `mark_bar(opacity=0.3)`. Auch hier verwenden wir das `tooltip`-Attribut, um per Mouseover Details zum jeweiligen Balken des Histogramms anzuzeigen.

In [ ]:
alt.Chart(df_aufnahmealter.query("kapitel_id.isin(['E', 'P'])")) \
  .mark_bar(opacity=0.3) \
  .encode(
    x = alt.X("aufnahmealter_jahre:Q", bin=alt.Bin(maxbins=40)),
    y = alt.Y("count()", stack=None),
    color = alt.Color("kapitel_name:N"),
    tooltip = alt.Tooltip(["kapitel_name", "count()"])
  )

### Boxplots

Boxplots dienen dazu, Verteilungen mehrerer gruppierter Daten übersichtlich mit Hilfe von Quantilen (typischerweise Quartilen) darzustellen. Die "Box" umfasst dabei jeweils 25% der Werte (Quartile) oberhalb und unterhalb des Medians (Strich in der Mitte der Box), während die Fehlerbalken Werte bis zum 1,5-fachen Interquartilsabstand umfassen. Werte außerhalb dieses Bereichs werden als "Ausreißer" durch kleine Kreise dargestellt.

Für das folgende Beispiel verwenden wir den gleichen Datensatz wie im Histogramm oben und setzen die Altair-Methode `mark_boxplot()` ein.

In [ ]:
alt.Chart(df_aufnahmealter.query("kapitel_id.isin(['E', 'P'])")) \
  .mark_boxplot() \
  .encode(
    x = alt.X("kapitel_id:O"),
    y = alt.Y("aufnahmealter_jahre:Q"),
    color = alt.Color("kapitel_name:N"),
    tooltip = alt.Tooltip(["kapitel_name", "count()"])
  )

## Kombination von Diagrammen

Eine Möglichkeit zur Kombination mehrere Diagramme haben wir oben beim Pie-Chart gezeigt (Pie-Chart und Beschriftungen wurden separat generiert & dann zusammengeführt). Auf dem gleichen Weg können auch verschiedene Diagrammtypen (z.B. Säulen- & Liniendiagramm) kombiniert werden.

An dieser Stelle soll als zusätzliche Möglichkeit noch gezeigt werden, ein Diagramm auf mehrere Teilgrafiken aufzuteilen, ohne dass dafür zunächst einzelne Abbildungen erzeugt & zusammengeführt werden müssen.

### Facetten

Mit "Facetten" kann ein Diagramm automatisch auf mehrere Teildiagramme aufgeteilt werden.

Hierfür laden wir zunächst eine weitere Aggregation von Fallzahlen nach Diagnosekapitel in einem Dataframe:

In [ ]:
# Fallzahlen zu den dokumentierten Diagnosen (ICD10-Kapitel) auswerten
sql = """
SELECT dkt.kapitel_id,
       dkt.kapitel_id || ' ' || dkt.kapitel_name AS kapitel_name,
       FLOOR(fal.aufnahmealter_jahre) AS aufnahmealter_jahre,
       COUNT(*)                       AS fallzahl
  FROM f_faelle   fal
  JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
 WHERE dkt.kapitel_name <> ''
 GROUP BY dkt.kapitel_id,
       dkt.kapitel_id || ' ' || dkt.kapitel_name,
       FLOOR(fal.aufnahmealter_jahre)
"""
resultset = %sql $db_url_reporting $sql
df_diagnosealter = resultset.DataFrame()
df_diagnosealter.head()

Im Encoding eines Charts kann mit dem `facet`-Attribut eine Spalte festgelegt werden, für deren Ausprägungen jeweils automatisch ein eigenes Diagramm erstellt wird. Altair ordnet diese Diagramme automatisch in einem Raster an. Als Option des `facet`-Attributs kann mit `rows` und/oder `columns` optional die Zahl der Zeilen & Spalten des Rasters festgelegt werden.

Hinweis: die Angaben zur Größe des Diagramms in der `properties()`-Methode beziehen sich bei einem facettierten Diagramm auf die Einzelgrafiken, nicht auf das zusammengefügte Gesamtdiagramm.

In [ ]:
alt.Chart(df_diagnosealter) \
  .mark_bar() \
  .encode(
    x = alt.X("aufnahmealter_jahre:Q", bin=alt.Bin(maxbins=40), title="Aufnahmealter"),
    y = alt.Y('fallzahl:Q', aggregate="sum", title="Fallzahl"),
    facet = alt.Facet("kapitel_id:O", columns=5),
    color = alt.Color("kapitel_name")
  ) \
  .properties(
    title  = "Altersverteilung nach Diagnosekapitel",
    width  = 90,
    height = 80
  )

## Geovisualisierung

Bei Geovisualisierung werden Daten auf Landkarten abgebildet. Die Daten können dabei über Längen- & Breitengrade einen expliziten Bezug zu einer geographischen Position mitbringen. Wenn nur ein indirekter Bezug über eine Adresse vorliegt, muss diese zunächst geocodiert werden. Entsprechende Services gibt es kostenpflichtig (z.B. von Google oder Microsoft), aber auch kostenfrei bzw. zum Betrieb auf einer eigenen Infrastruktur (z.B. OpenStreetMap Nominatim).

🛑 Caveat: Exakte Adressen (z.B. von Patienten) sind in der Regel als personenbezogene Daten anzusehen und sollten daher nicht ohne weiteres über einen externen Service geocodiert werden, da hierbei ggf. eine unerwünschte Weitergabe von personenbezogenen Informationen an Dritte erfolgt. Es sollte entweder ein lokal installierter Geocoding-Service (z.B. Nominatim) genutzt oder eine vergröberte Form der Adrese (z.B. PLZ, Gemeindekennziffer) genutzt werden.

Für die Geovisualisierung laden wir zunächst die im Kurs-DWH schon vorhandenen Ortsangaben (mit Längen- & Breitengrad) der (synthetischen) Patienten und Versorgungseinrichtungen:

In [ ]:
# Geokoordinaten der Patient:innen auslesen
sql = """
SELECT patient_id,
       patient_nachname || ', ' || patient_vorname AS patient_name,
       patient_lat,
       patient_lon
  FROM d_patient
"""
resultset = %sql $db_url_reporting $sql
df_patienten = resultset.DataFrame()
df_patienten.head()

In [ ]:
# Geokoordinaten der Versorgungseinrichtungen & ihre Fallzahl auslesen
sql = """
SELECT org.einrichtung_id,
       org.einrichtung_name,
       org.einrichtung_lat,
       org.einrichtung_lon,
       COUNT(*) AS fallzahl
  FROM      d_orga org
  LEFT JOIN f_faelle fal ON org.einrichtung_id = fal.einrichtung_id
 GROUP BY org.einrichtung_id,
       org.einrichtung_name,
       org.einrichtung_lat,
       org.einrichtung_lon
"""
resultset = %sql $db_url_reporting $sql
df_einrichtungen = resultset.DataFrame()
df_einrichtungen.head()

Für die interaktive Geovisualisierung nutzen wir das *Folium*-Package. Es stellt die JavaScript-basierte Leaflet-Bibliothek in Python zur Verfügung, mit der verschiedene Basiskarten (z.B. OpenStreetMap) als Hintergrund genutzt und eine Reihe von Markern oder auch Polygonen darauf platziert werden können.

Mit der Methode `folium.Map()` wird zunächst die Basiskarte erzeugt. Hierbei sollte eine Geokoordinate mit Längen-/Breitengrad sowie ein Zoom-Faktor für den initial anzuzeigenden Kartenausschnitt angegeben werden. Die Karte kann in dieser Form bereits angezeigt und per Pan & Zoom verändert werden.

Mit der Methode `folium.CircleMaker` können Kreise auf der Karte platziert werden. Hierfür iterieren wir mit der Methode `itertuples()` über die Zeilen des Dataframes und setzen die Längen-/Breitengradangaben sowie ggf. Größenangaben als Parameter ein. Optional kann wieder ein `tooltip`-Parameter gesetzt werden, um per Mouseover-Effekt Details zu einem Marker anzuzeigen. Die Methode `add_to()` fügt den jeweils erzeugten Marker in die Basiskarte ein.

In [ ]:
# Basiskarte erzeugen und Mittelpunkt/Zoomfaktor auf Anzeige des gesamten Bundesgebiets setzen
map = folium.Map(location=[51.287819, 10.099264], zoom_start=6)

# Über Patienten iterieren und für jeden Patienten einen CircleMaker mit Größe 2 in rot einsetzen
for row in df_patienten.itertuples():
    folium.CircleMarker(location=[row.patient_lat,row.patient_lon],
                        radius=2,
                        color="red",
                        fill=True,
                        opacity=0.5,
                        tooltip=row.patient_name).add_to(map)

# Über Versorgungseinrichtungen iterieren und jeweils einen CircleMaker mit fallzahlbezogener Größe in blau einsetzen
for row in df_einrichtungen.itertuples():
    folium.CircleMarker(location=[row.einrichtung_lat,row.einrichtung_lon],
                        radius=row.fallzahl/250,
                        color="blue",
                        fill=True,
                        opacity=0.5,
                        tooltip=row.einrichtung_name + f" ({row.fallzahl})").add_to(map)

# Karte anzeigen
map